<a href="https://colab.research.google.com/github/zskylarli/transcrobialGAN/blob/main/transcrobial_pix2pix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transcrobial GAN

*PyTorch implementation*

Last updated: 4/14/2021

Author: Skylar Li 

Used datasets:

- /finalproject (680 images)

> DIBaS Sorted Dataset for pix2pix [https://drive.google.com/drive/folders/1sRcxDldxM6WQz1JjvO_IqI375obUv6wC?usp=sharing]




# Install

In [ ]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [ ]:
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [ ]:
!pip install -r requirements.txt

# Dataset

We uploaded the DIBaS dataset to a shared Google Drive folder. 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

In [ ]:
PATH_OF_DATA= '/content/gdrive/"My Drive"/finalproject'
!ls {PATH_OF_DATA}

In [ ]:
!python ./datasets/combine_A_and_B.py --fold_A /content/gdrive/"My Drive"/finalproject/A --fold_B /content/gdrive/"My Drive"/finalproject/B --fold_AB /content/gdrive/"My Drive"/finalproject/AB

# Training

-   `python train.py --dataroot /content/gdrive/"My Drive"/finalproject/AB --name dibas_pix2pix --model pix2pix --direction BtoA`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. Add `--direction BtoA` if you want to train a model to transfrom from class B to A.

In [ ]:
python train.py --dataroot /content/gdrive/"My Drive"/finalproject/AB --name dibas_pix2pix --model pix2pix --direction BtoA

# Testing

-   `python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_pix2pix`

Change the `--dataroot`, `--name`, and `--direction` to be consistent with your trained model's configuration and how you want to transform images.


In [ ]:
!ls checkpoints/

In [ ]:
python test.py --dataroot /content/gdrive/"My Drive"/finalproject/AB --direction BtoA --model pix2pix --name dibas_pix2pix

# Save results to My Drive

In [ ]:
!zip -r /content/gdrive/"My Drive"/finalproject/pix2pix_results.zip /content/pytorch-CycleGAN-and-pix2pix/results/dibas_pix2pix/test_latest/images